# HilltopAds

# Задание

В архиве test.zip находятся файлы:
- requests.csv – лог запросов за одну минуту (время до секунд; наносекунды; название фида; IP; User Agent); Фиды – источники трафика, сущности, по которым нужно получить описываемые далее метрики.
- stat_rps.tsv – выгрузка статистики по каждому фиду с суммарным дневным числом запросов и максимальным значением RPS (requests per second).

Необходимо составить отчет в виде .tsv файла (report.tsv), содержащего таблицу со следующими столбцами:
- Feed – название фида;
- Total – суммарное число запросов в логе, относящихся к фиду;
- Uniq – число уникальных пар IP+UserAgent в логе для данного фида;
- MaxRPS – максимальное значение RPS для данного фида;
- Requests – суммарное число запросов за день, относящихся к данному фиду;
- Resell – процент уникальных пар IP+UserAgent данного фида, которые впервые (по времени) были замечены у другого фида в представленном логе;
- UniquenessMean | UniquenessMedian | UniquenessPerc90 – среднее значение уникальности пары IP+UserAgent, медиана и 90-й процентиль. 

Уникальность – количество повторений пары IP+UserAgent в логе: 1, 2, 3,.... Для фида уникальность конкретной пары считается как число ее глобального появления в логе на момент ее последнего появления в этом фиде.

Пример:
- В 12:00:00.325436 был запрос с параметрами IP+UserAgent [51.133.3.12, Mozilla/5.0 Galeon/1.2.9 (X11; Linux i686; U;) Gecko/20030319 Debian/1.2.9-0.0.3], относящийся к фиду feed1.
- Позже в 12:00:00.5463878 запрос с этими же параметрами [51.133.3.12, Mozilla/5.0 Galeon/1.2.9 (X11; Linux i686; U;) Gecko/20030319 Debian/1.2.9-0.0.3] был зафиксирован для фида feed2.
- В 12:00:02 запрос [51.133.3.12, Mozilla/5.0 Galeon/1.2.9 (X11; Linux i686; U;) Gecko/20030319 Debian/1.2.9-0.0.3] снова относился к feed1.
- Кроме этих трех запросов для указанных фидов было еще по 2 глобально уникальных запроса.
- Других запросов для этих или других фидов за минуту не было. Суммарно в логе 7 запросов.
- Получаем:
    - Уникальности пары [51.133.3.12, Mozilla/5.0 Galeon/1.2.9 (X11; Linux i686; U;) Gecko/20030319 Debian/1.2.9-0.0.3] для фидов feed1 и feed2 равны 3 и 2 соответственно. У всех других пар IP+UserAgent уникальности равны 1.
    - Для feed2 зафиксирован реселл, у feed1 реселла нет, так как пара [51.133.3.12, Mozilla/5.0 Galeon/1.2.9 (X11; Linux i686; U;) Gecko/20030319 Debian/1.2.9-0.0.3] изначально относилась к нему.
    - С учетом всех семи запросов имеем следующие метрики:
        - feed1: Resell = 0/3 = 0;  UniquenessMean = (3 + 1 + 1)/3 = 1.666. Здесь у feed1 четыре запроса, но уникальных пар IP+UserAgent – три.
        - feed2: Resell = 1/3 = 0.333;  UniquenessMean = (2 + 1 + 1)/3 = 1.333

Требования:
- строки должны быть отсортированы по столбцу Total (desc);
- разделитель столбцов – символ табуляции '\t';
- все числовые данные сохраняются как числа с плавающей точкой без округлений;
- процентные значения сохраняются как числа из интервала [0; 1]: писать знак % и умножать на 100 не нужно;
- первая строка файла должна содержать названия столбцов;
- порядок и названия столбцов должны в точности соответствовать заданию; никаких дополнительных столбцов быть не должно;
- вместе с отчетом нужно предоставить исходный код, использовавшийся для его формирования, и инструкции для запуска.

# Реализация

In [1]:
from functions import *

### Вид лога

In [2]:
logs = load_logs()
for i in logs:
    print(i)
    break

['12:00:00', '672693', 'feed0', '157.44.215.151', 'Mozilla/5.0 (Linux; Android 9; vivo 1907) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.93 Mobile Safari/537.36']


### Вид дневной статистики

In [3]:
with ZipFile('test.zip') as z:
    stat = pd.read_csv(z.open('stat_rps.tsv'), sep='\t')
stat.head(2)

,feed,requests,rps
0,feed79,3155278,66.913611
1,feed275,20363,2.053333


### Вид на выходе

In [4]:
report = pd.read_csv('report.tsv', '\t')
report.head(2)

,Feed,Total,Uniq,MaxRPS,Requests,Resell,UniquenessMean,UniquenessMedian,UniquenessPerc90
0,feed231,587959,492876,32747.403056,2.391022e+09,0.096156,1.389836,1.0,2.0
1,feed227,271842,247841,39622.197778,2.797295e+09,0.213855,1.692206,1.0,3.0
